<img src="http://xanadu-img.s3-website-us-east-1.amazonaws.com/logo.png" width=70%>

---

<center> <h1> Boson sampling tutorial </h1></center>

To see how to construct and simulate a continuous-variable quantum circuit in Strawberry Fields, let’s consider the case of [boson sampling](https://en.wikipedia.org/wiki/Boson_sampling). 

## Introduction to boson sampling
---

Introduced by [Aaronson and Arkhipov](https://doi.org/10.4086/toc.2013.v009a004), boson sampling presented a slight deviation from the general approach in quantum computation. Rather than presenting a theoretical model of universal quantum computation (i.e. a framework that enables quantum simulation of any arbitrary Hamiltonian), boson sampling-based devices are instead an example of an **intermediate quantum computer**, designed to experimentally implement a computation that is classically computationally hard.

Boson sampling proposes the following [quantum linear optics](https://en.wikipedia.org/wiki/Linear_optical_quantum_computing) scheme. An array of single photon sources are set up, designed to simultaneously emit single photon states into a multimode linear [interferometer](https://en.wikipedia.org/wiki/Interferometry); the results are then generated by sampling from the probability of single photon measurements from the output of the linear interferometer.
<img src="http://3.bp.blogspot.com/-EQpvowD7Aps/UTN0nUBrlHI/AAAAAAAACsQ/5_ebFrLcWmo/s0/Boson+sampling+fig1.png" width=50%/>
<center><em>
<a href="http://www.2physics.com/2013/03/experimental-boson-sampling.html">Steve Kolthammer, Justin Spring, Ian Walmsley, Department of Physics, University of Oxford</a></em></center>

For example, consider $N$ single photon Fock states, $$|\psi\rangle=|m_1,m_2,\dots,m_N\rangle,$$ composed of $b=\sum_i m_i$ photons, incident on an $N$-mode linear interferometer, which performs the following linear transformation of the input mode creation and annihilations operators:

$$\begin{align}
    &\hat{a}^\dagger_{out_k} = \sum_{j=0}^N U_{kj}\hat{a}^\dagger_{in_j},~~~~\hat{a}_{out_k} = \sum_{j=0}^N U_{jk}^\dagger\hat{a}_{in_j}
\end{align}$$


Here, the unitary $U$ completely describes the interferometer. We will denote the output state of the linear interferometer as $|\psi'\rangle$ - thus, the probability of detecting $n_j$ photons at the $j$th output mode is given by
$$\left|\left\langle n_1,n_2,\dots,n_N\middle|{\psi'}\right\rangle\right|^2$$

The remarkable nature of the boson sampling problem becomes clear when you consider that

$$\left|\left\langle{n_1,n_2,\dots,n_N}\middle|{\psi'}\right\rangle\right|^2 = \frac{\left|\text{Per}(U_{st})\right|^2}{m_1!m_2!\cdots m_N!n_1!n_2!\cdots n_N!}$$

i.e. the sampled single photon probability distribution is proportional to the **permanent** of $U_{ST}$, a submatrix of the interferometer unitary, dependent upon the input and output Fock states.

<div class="alert alert-success" style="border: 0px; border-left: 3px solid #119a68; color: black; background-color: #daf0e9">

<p style="color: #119a68;">**The Permanent**</p>

The [permanent of a matrix](https://en.wikipedia.org/wiki/Permanent), defined by

$$\text{Per}(A) = \sum_{\sigma=S_N}\prod_{i=1}^N A_{i\sigma(i)},$$

where $S_N$ is the set of all permutations of $N$ elements, is calculated in a similar fashion to the determinant, but unlike the determinant, the signatures of the permutations are not taken into account - every permutation is taken as a positive quantity.

In graph theory, the permanent calculates the number of perfect <a href="https://en.wikipedia.org/wiki/Matching_(graph_theory)">matchings</a> in a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph>) with adjacency matrix $A$.
    
</div>

Whilst the determinant can be calculated efficiently on classical computers, [calculation of the permanent](https://en.wikipedia.org/wiki/Computing_the_permanent) belongs to the computational complexity class #P (<a href="https://doi.org/10.1016/0304-3975(79)90044-6">Valiant, 1979</a>), which is strongly believed to be classically hard to calculate. (Surprisingly, even calculating the permanent in an *approximate* manner is a member of #P and classically hard). 

Yet, the permanent of (some) matrix can be computed efficiently in the boson sampling scheme, providing a potential challenge to classical computation, and demonstrating the power of quantum computation.

## Continuous-variable implementation
---

In quantum linear optics, the multimode linear interferometer is commonly decomposed into two-mode beamsplitters (`BSgate`) and single-mode phase shifters (`Rgate`) (<a href="https://doi.org/10.1103/physrevlett.73.58">Reck, 1994</a>), allowing for an almost trivial translation into a continuous-variable quantum circuit.

For example, in the case of a 4 mode interferometer, with arbitrary $4\times 4$ unitary $U$, the continuous-variable quantum circuit is given by

<img src="https://s3.amazonaws.com/xanadu-img/boson_sampling.svg" width=70%/>

In the above,
* the detectors perform Fock state measurements,
* the parameters of the beamsplitters and the rotation gates determines the unitary $U$.

Note that, in order to allow for arbitrary linear unitaries for $m$ imput modes, we must have a minimum of $m+1$ columns in the beamsplitter array ([Clements, 2016](https://arxiv.org/abs/1603.08788)). 



## Simulating boson sampling in Strawberry Fields
---

To see how to simulate a boson sampling circuit using the above circuit diagram, let's walk through the construction of a Strawberry Fields program. 

### Importing Strawberry Fields

The first thing we need to do is import Strawberry Fields; we do this with the following import statements:

In [1]:
import strawberryfields as sf
from strawberryfields.ops import *

The first import statement imports Strawberry Fields as `sf`, allowing us to access the engine and backends. The second import statement imports all available continuous-variable gates into the global namespace.

We'll also need to import $\pi$, either from the `math` module or `numpy`:

In [2]:
from numpy import pi

### Engine initialisation

We can now initialise our chosen backend, engine, and quantum register, using the 
`strawberryfields.new_engine()` function, which has the following syntax:

    sf.Engine(num_subsystems, reset=True, *args, **kwargs)

where

* `num_subsystems` (*int*) is the number of modes we want to initialise in our quantum register

* ``reset`` is a boolean that instructs the engine to begin the simulation with all modes reset to the vacuum state. If ``reset=False``, then this is skipped, and the engine will be run using the output simulation state from any previous engine runs.

Now that that's all sorted, time to initialise our engine:

In [3]:
eng, q = sf.Engine(4)

The `new_engine` function returns two objects:
* `eng`, the quantum engine itself, and
* `q`, our 4 mode quantum register

The register is returned as a *tuple*, with each individual mode accessed via indexing. For example, `q[0]` represents the first mode.

Let's get to building our boson sampling circuit.

### Circuit construction
To prepare states and apply gates, we must be inside the context of the engine we initialised, using the `with` statement, and using the notation

    Operator(args) | mode(s)

with each successive operator applied in the correct, temporal order. 

For example, to construct the boson sampling circuit

<img src="https://s3.amazonaws.com/xanadu-img/boson_sampling.svg" width=70%/>

we would run:

In [4]:
with eng:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac | q[2]
    Fock(1) | q[3]

    # rotation gates
    Rgate(0.5719) | q[0]
    Rgate(-1.9782) | q[1]
    Rgate(2.0603) | q[2]
    Rgate(0.0644) | q[3]

    # beamsplitter array
    BSgate(0.7804, 0.8578) | (q[0], q[1])
    BSgate(0.06406, 0.5165) | (q[2], q[3])
    BSgate(0.473, 0.1176) | (q[1], q[2])
    BSgate(0.563, 0.1517) | (q[0], q[1])
    BSgate(0.1323, 0.9946) | (q[2], q[3])
    BSgate(0.311, 0.3231) | (q[1], q[2])
    BSgate(0.4348, 0.0798) | (q[0], q[1])
    BSgate(0.4368, 0.6157) | (q[2], q[3])

This is a fair bit to process, so let's go through what's actually happening.

1. To begin with, we initialise the input state $|1,1,0,1\rangle$, by creating a single photon Fock state in modes 0, 1 and 3.
    * This is done via the `Fock()` operator.
<br><br>

2. Mode 2 is initialised as a vacuum state using the `Vac` operator.
    * This is *optional* - modes are automatically created in the vacuum state upon engine initialisation.
<br><br>

3. Next we apply the rotation gates, `Rgate(p)`, to each mode.
    * The resulting rotation in the phase space occurs in the anticlockwise direction, with angle $\phi=2\pi p$.
<br><br>
    
4. Finally, we apply the array of beamsplitters, using the `BSgate(q,p)` operator.
    * The arguments are related to the beamsplitter angles by the relationships $\theta = 2\pi q$ and $\phi=2\pi p$.
    * The transmission amplitude is then given by $t=\cos\theta$
    * The reflection amplitude is given by $r=e^{i\phi}\sin{\theta}$
    
Note that, in the above Strawberry Fields circuit, we have chosen **random** values for the rotation gates and the beamsplitter array.

<div class="alert alert-success" style="border: 0px; border-left: 3px solid #119a68; color: black; background-color: #daf0e9">
<p style="color: #119a68;">**Aside**</p>
Strawberry Fields gates are standard Python objects, and can be treated as such. For example, if we had needed to reuse a gate with the same parameters multiple times, we could write the following:
<br>
<pre style="background-color: #daf0e9">
with eng:
    BS = BSgate(0.12,0.543)
    BS | (q[0], q[1])
    BS | (q[1], q[2])
</pre>
</div>

### Running the engine
Once the circuit is constructed, you can run the engine via the `eng.run()` method:

In [5]:
state =eng.run('fock', cutoff_dim=5)

where

* `backend` (*str*) represents the Strawberry Fields backend we wish to use; we have the choice of two Fock backends, Fock (`'fock'`) and Tensorflow (`'tf'`), and one gaussian backend (`'gaussian'`).

<div class="alert alert-success" style="border: 0px; border-left: 3px solid #119a68; color: black; background-color: #daf0e9">
<p style="color: #119a68;">**The Strawberry Fields backends**</p>

<ul>
<li>**Fock backends** are backends which represent the quantum state and operations via the Fock basis. <br>These can represent all possible continuous-variable states and operations, but also introduce numerical error due to truncation of the Fock space, and can consume more memory.</li>
<br>

<li>The **gaussian backend**, due to its ability to represent states and operations as gaussian objects/transforms in the phase space, can be less computationally intensive then the Fock backends. <br>However, it cannot represent non-gaussian operations and states (such as the cubic phase gate, fock measurements, and fock states, amongst others).</li>
</ul>
</div>

Since boson sampling requires single photon Fock states as input, we need to use a Fock backend; let's choose the standard Fock backend for this particular example.

Since we are using a Fock backend, we must also specify the Fock basis *cutoff dimension*; let's choose `cutoff_dim=5`, such that a state $|\psi\rangle$ has approximation

$$|{\psi}\rangle = \sum_{n=0}^\infty c_n|{n}\rangle \approx\sum_{n=0}^{5-1} c_n|{n}\rangle$$

in our truncated Fock basis - i.e. all information of quantum amplitudes of Fock states $|n\rangle$, with $n\geq 5$, is discarded.


<div class="alert" style="border: 0px; border-left: 3px solid #F0AD4E; color: black; background-color: #FFF1E3">
<p style="color: #9B5831;">**Warning**</p>
To avoid significant numerical error when working with the Fock backend, we generally need to ensure that all initial states and gates we apply result in negligible amplitude in the Fock basis for Fock states $|{n}\rangle, ~~n\geq \texttt{cutoff_dim}$.
<br><br>

For example, to prepare a squeezed vacuum state in the $x$ quadrature with `cutoff_dim=10`, a squeezing factor of $r=1$ provides an acceptable approximation, since $\left|\left\langle{n}\middle|{z}\right\rangle\right|^2<0.02$ for $n\geq 10$.
</div>

<div class="alert alert-success" style="border: 0px; border-left: 3px solid #119a68; color: black; background-color: #daf0e9">
<p style="color: #119a68;">**Scaling**</p>
As we are performing a classical simualation of a quantum system, we still need to consider how the simulation scales. Using the Fock backends, memory usage and computational time will scale like $\sim D^N$, where $N$ is the number of modes and $D$ the cutoff dimension.
<br><br>
Ideally, we would like to choose a cutoff large enough that minimises truncation error, but small enough to reduce classical computational resources!
</div>

Note that `eng.run` can either go inside or outside the engine context used earlier, and will execute all operations within the command queue. 

You can even continue to queue operations after running the engine, allowing you to break your circuit execution into multiple command queues and engine runs.

Other useful methods that can be called at any time include:

* `eng.print_queue()`: print the command queue (the operations to be applied on the next call to `eng.run`)
* `eng.print_applied()`: prints all commands applied on all prior calls to `eng.run` since the backend was last reset.
* `eng.reset_queue()`: clear all operations from the command queue.
* `eng.reset_backend()`: retains the command queue, but resets all modes to the vacuum state.
* `eng.reset()`: clear all operations from the command queue, *and* resets all modes to the vacuum state.

For example, try running the following:

In [6]:
eng.print_applied()

Fock(1), 	(reg[0])
Fock(1), 	(reg[1])
Vac, 	(reg[2])
Fock(1), 	(reg[3])
Rgate(0.5719), 	(reg[0])
Rgate(-1.978), 	(reg[1])
Rgate(2.06), 	(reg[2])
Rgate(0.0644), 	(reg[3])
BSgate(0.7804, 0.8578), 	(reg[0], reg[1])
BSgate(0.06406, 0.5165), 	(reg[2], reg[3])
BSgate(0.473, 0.1176), 	(reg[1], reg[2])
BSgate(0.563, 0.1517), 	(reg[0], reg[1])
BSgate(0.1323, 0.9946), 	(reg[2], reg[3])
BSgate(0.311, 0.3231), 	(reg[1], reg[2])
BSgate(0.4348, 0.0798), 	(reg[0], reg[1])
BSgate(0.4368, 0.6157), 	(reg[2], reg[3])


## Calculating the interferometer unitary
---

Before we get into analysing the results of the Strawberry Fields simulation, let's calculate the unitary operator described by the rotation gates and beamsplitters - this gives us something to compare it to later.

For this part, we'll need the NumPy and SciPy libraries:

In [7]:
import numpy as np
from numpy.linalg import multi_dot
from scipy.linalg import block_diag

### Rotation gate unitary

To start with, let's generate the matrix representing the unitary transformation of the input mode annihilation and creation operators. The rotation gates simply act as follows,

$$R(\phi)\hat{a} = \hat{a}e^{i\phi},$$

and thus the column of rotation gates has the following block diagonal form:

$$\begin{align}
    U_\phi = \left[\begin{matrix}
            e^{i\phi_1} & 0 & \cdots\\
            0 & e^{i\phi_2}  & \cdots \\
            \vdots & \vdots & \ddots \\
        \end{matrix}\right]
\end{align}$$

In [8]:
Uphase = np.diag([np.exp(0.5719*1j),np.exp(-1.9782*1j),np.exp(2.0603*1j),np.exp(0.0644*1j)])

In [9]:
np.round(Uphase,4)

array([[ 0.8409+0.5412j,  0.0000+0.j    ,  0.0000+0.j    ,  0.0000+0.j    ],
       [ 0.0000+0.j    , -0.3962-0.9182j,  0.0000+0.j    ,  0.0000+0.j    ],
       [ 0.0000+0.j    ,  0.0000+0.j    , -0.4702+0.8826j,  0.0000+0.j    ],
       [ 0.0000+0.j    ,  0.0000+0.j    ,  0.0000+0.j    ,  0.9979+0.0644j]])

### Beamsplitter unitary

A single beamsplitter, acting on two input modes $(\hat{a}_1,\hat{a}_2)$, instead acts as follows:

$$\begin{align}
BS(\theta, \phi) (\hat{a}_1,\hat{a}_2) = \left[\begin{matrix}
            t & -r^*\\
            r & t\\
        \end{matrix}\right] \left[\begin{matrix}
            \hat{a}_1\\
            \hat{a}_2
        \end{matrix}\right]
\end{align}$$

where $t=\cos(\theta)$ and $r=e^{i\phi}\sin(\theta)$. Again, like, the rotation gate, beamsplitters in the same 'column' (i.e. acting on independent modes) combine as block diagonal matrices.

First of all, we need to convert the `BSgate` arguments, `q` and `p` (reproduced below for convenience),

In [10]:
BSargs = [
    (0.7804, 0.8578),
    (0.06406, 0.5165),
    (0.473, 0.1176),
    (0.563, 0.1517),
    (0.1323, 0.9946),
    (0.311, 0.3231),
    (0.4348, 0.0798),
    (0.4368, 0.6157)
]

into transmission and reflection amplitudes $t$ and $r$:

In [11]:
t_r_amplitudes = []

for q,p in BSargs:
    t = np.cos(q)
    r = np.exp(1j*p)*np.sin(q)
    t_r_amplitudes.append((t,r))

In [12]:
t_r_amplitudes

[(0.71063216937901652, (0.4602032449982007+0.53217938060093872j)),
 (0.99794885977982051, (0.05566547093254029+0.031613740841832737j)),
 (0.89020561932289122, (0.45241244157971522+0.053450332364058703j)),
 (0.84565774216485023, (0.52759605355941086+0.080655981707390259j)),
 (0.99126111276958639, (0.071872018569631851+0.11061563749094347j)),
 (0.95202803513336776, (0.29017650247396554+0.097160782891430389j)),
 (0.9069543062416473, (0.41988852889473471+0.033578411105169972j)),
 (0.90611003485741559, (0.34535860197634749+0.24431954643792997j))]

Now, we can generate the unitary matrix representation for each individual beamsplitter:

In [13]:
BSunitaries = [np.array([[t, -np.conj(r)], [r, t]]) for t,r in t_r_amplitudes]

before using the SciPy function `scipy.linalg.block_diag` to calculate the overall beamsplitter unitary, $U_{BS_i}$, for *each 'column'* of independent beamsplitters:

In [14]:
# first column (2 beamsplitters)
UBS1 = block_diag(*BSunitaries[0:2])

# second column (1 beamsplitters)
UBS2 = block_diag([[1]], BSunitaries[2], [[1]])

# third column (2 beamsplitters)
UBS3 = block_diag(*BSunitaries[3:5])

# fourth column (1 beamsplitters)
UBS4 = block_diag([[1]], BSunitaries[5], [[1]])

# fifth column (2 beamsplitters)
UBS5 = block_diag(*BSunitaries[6:8])

Finally, we can combine the unitaries to form a single $4\times 4$ unitary via matrix multiplication;

$$U = U_{BS_5}U_{BS_4}U_{BS_3}U_{BS_2}U_{BS_1}U_{\phi}.$$

Since `numpy.dot` only supports matrix multiplication of two arrays, we instead use `numpy.linalg.multi_dot`:

In [15]:
U = multi_dot([UBS5, UBS4, UBS3, UBS2, UBS1, Uphase])

In [16]:
print(U)

[[ 0.21954694-0.25653455j  0.61107685+0.52417894j -0.10270019+0.47447883j
  -0.02725023+0.03729095j]
 [ 0.45128186+0.60258291j  0.45695259+0.01230749j  0.13162587-0.45041774j
   0.03528319-0.05324427j]
 [ 0.03871009+0.49271556j -0.01921274-0.32184285j -0.24077647+0.52443283j
  -0.45838814+0.32963337j]
 [-0.15661908+0.22456857j  0.10999222-0.16375022j -0.42117984+0.18364484j
   0.81876918+0.06801566j]]


In [17]:
print(np.round(U,4))

[[ 0.2195-0.2565j  0.6111+0.5242j -0.1027+0.4745j -0.0273+0.0373j]
 [ 0.4513+0.6026j  0.4570+0.0123j  0.1316-0.4504j  0.0353-0.0532j]
 [ 0.0387+0.4927j -0.0192-0.3218j -0.2408+0.5244j -0.4584+0.3296j]
 [-0.1566+0.2246j  0.1100-0.1638j -0.4212+0.1836j  0.8188+0.068j ]]


Examining the output of the above cell, you should have the result

$$\begin{equation}
    U = \left[\begin{matrix}
        0.2195-0.2565i & 0.6111+0.5242i & -0.1027+0.4745i & -0.0273+0.0373i\\
        0.4513+0.6026i & 0.4570+0.0123i & 0.1316-0.4504i & 0.0353-0.0532i\\
        0.0387+0.4927i & -0.0192-0.3218i & -0.2408+0.5244i & -0.4584+0.3296i\\
        -0.1566+0.2246i & 0.1100-0.1638i & -0.4212+0.1836i & 0.8188+0.068i
    \end{matrix}\right]
\end{equation}$$

This represents the unitary transformation of the input creation and annihilation operators of the linear interferometer.

## Analysis
---

Now that we have the interferometer unitary transformation $U$, as well as the simulation results, let's compare the two, and see if the boson sampling result,

$$\left|\left\langle{n_1,n_2,\dots,n_N}\middle|{\psi'}\right\rangle\right|^2 = \frac{\left|\text{Perm}(U_{st})\right|^2}{m_1!m_2!\cdots m_N!n_1!n_2!\cdots n_N!},$$

holds.

### Extracting results from Strawberry Fields

First, let's extract the joint Fock state probabilities from our Strawberry fields simulation:

In [19]:
probs = state.all_fock_probs()

Here, the function `all_fock_state_probs()` returns a size $5\times 5\times 5\times 5$ array (due to having 4 particles with cutoff dimension 5) containing the Fock state probabilities of the output state,

$$\left\langle i_1,i_2,\dots,i_N \middle| {\psi'} \middle| j_1,j_2,\dots,j_N\right\rangle$$

For example, the probability $\left|\left\langle{2,0,0,1}\middle |{\psi'}\right\rangle\right|^2$ can be extracted via

In [20]:
probs[2,0,0,1]

0.1064419272464233

### Comparing SF result to the Permanent

Before we can calculate the right hand side of the boson sampling equation, we need a method of calculating the permanent. Since the permanent is classically hard to compute, it is not provided in either NumPy *or* SciPy, so we will use the definition provided in [SciPy Issue #7151](https://github.com/scipy/scipy/issues/7151) on GitHub: 

In [21]:
def perm(M):
    n = M.shape[0]
    d = np.ones(n)
    j =  0
    s = 1
    f = np.arange(n)
    v = M.sum(axis=0)
    p = np.prod(v)
    while (j < n-1):
        v -= 2*d[j]*M[j]
        d[j] = -d[j]
        s = -s
        prod = np.prod(v)
        p += s*prod
        f[0] = 0
        f[j] = f[j+1]
        f[j+1] = j+1
        j = f[0]
    return p/2**(n-1) 

<div class="alert alert-success" style="border: 0px; border-left: 3px solid #119a68; color: black; background-color: #daf0e9">
<p style="color: #119a68;">**Note**</p>
This function makes use of the [Balasubramanian-Bax-Franklin-Glynn formula](https://en.wikipedia.org/wiki/Computing_the_permanent#Balasubramanian-Bax-Franklin-Glynn_formula), which scales as $\mathcal{O}(2^{n-1}n^2)$ for an $n\times n$.
</div>

Finally, how do we determine the submatrix $`U_{st}$? This isn't too hard ([Tillmann, 2013](https://doi.org/10.1038/nphoton.2013.102)); first, we calculate the submatrix $U_s$ by taking $m_k$ copies of the $k$th **columns** of $U$, where $m_k$ is the photon number of the $k$th input state.

Since our input state is $\left|{\psi}\right\rangle=\left|{1,1,0,1}\right\rangle$, we simply take single copies of the first, second, and fourth columns:

In [22]:
U[:,[0,1,3]]

array([[ 0.21954694-0.25653455j,  0.61107685+0.52417894j,
        -0.02725023+0.03729095j],
       [ 0.45128186+0.60258291j,  0.45695259+0.01230749j,
         0.03528319-0.05324427j],
       [ 0.03871009+0.49271556j, -0.01921274-0.32184285j,
        -0.45838814+0.32963337j],
       [-0.15661908+0.22456857j,  0.10999222-0.16375022j,
         0.81876918+0.06801566j]])

Next, we take $n_k$ copies of the $k$\text{th} **row** of $U_s$, where $n_k$ is the photon number of the $k$th **output** state that is measured. 

Here, we our measuring probability of of detecting the output state $\left|{2,0,0,1}\right\rangle$ so we take two copies of the first row, and one copy of the last row:

In [23]:
U[:,[0,1,3]][[0,0,3]]

array([[ 0.21954694-0.25653455j,  0.61107685+0.52417894j,
        -0.02725023+0.03729095j],
       [ 0.21954694-0.25653455j,  0.61107685+0.52417894j,
        -0.02725023+0.03729095j],
       [-0.15661908+0.22456857j,  0.10999222-0.16375022j,
         0.81876918+0.06801566j]])

Now, using the permanent function we defined above, we can take the absolute value squared of the permanent, and divide by the product of the input and output state number factorials, as per the boson sampling equation.

Since $0!=1!=1$, we only need to take into account the case $2!=2$:

In [24]:
np.abs(perm(U[:,[0,1,3]][[0,0,3]]))**2 / 2

0.10644192724642332

Comparing this to the result from Strawberry Fields,

In [25]:
BS = np.abs(perm(U[:,[0,1,3]][[0,0,3]]))**2 / 2
SF = probs[2,0,0,1]

100*np.abs(BS-SF)/BS

1.3037896031031119e-14

**They agree with almost negligable error!** This is due to the high accuracy of the Fock backend, despite the Fock state truncation/cutoff.

<div class="alert alert-success" style="border: 0px; border-left: 3px solid #119a68; color: black; background-color: #daf0e9">
<p style="color: #119a68;">**Note**</p>

Due to the Fock basis truncation/cutoff dimension, the Fock backends only return an **approximate** of the Fock state joint probability, and thus only approximate the various submatrix permanents.
<br>

However, the Fock backends are still computing a classically intractable problem, as even approximating the matrix permanent remains a hard classical problem.
</div>

In [26]:
np.abs(perm(U[:,[0,1,3]][[0,0,0]]))**2 / 6

0.00094584833471324887

In [27]:
>>> probs[3,0,0,0]

0.0009458483347132479

In [28]:
>>> np.abs(perm(U[:,[0,1,3]][[0,1,3]]))**2 / 1

0.17468916048563934

In [29]:
>>> probs[1,1,0,1]

0.17468916048563926